## Imports

In [64]:
import numpy as np
import pandas as pd
from rectools.models import PopularModel
from rectools.dataset import Dataset
from rectools import Columns
from rectools.metrics import Precision, Recall, MAP, MeanInvUserFreq, Serendipity, calc_metrics

## Data

In [2]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')

In [3]:
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [4]:
max_date = interactions[Columns.Datetime].max().normalize()
max_date

Timestamp('2021-08-22 00:00:00')

## Train

In [14]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=None
)

In [25]:
popular_model = PopularModel()
popular_model.fit(dataset);

In [66]:
pop = popular_model.recommend(
    users=test[Columns.User].unique(),
    dataset=dataset,
    k=100,
    filter_viewed=False)

In [51]:
popular_model_mean_weights = PopularModel(popularity="mean_weight")
popular_model_mean_weights.fit(dataset);

In [65]:
pop_mean_weights = popular_model_mean_weights.recommend(
 users=test[Columns.User].unique(),
    dataset=dataset,
    k=100,
    filter_viewed=False)

In [53]:
pop.head(10)

,user_id,item_id,score,rank,title
0,176549,10440,202457.0,1,Хрустальный
1,176549,15297,193123.0,2,Клиника счастья
2,176549,9728,132865.0,3,Гнев человеческий
3,176549,13865,122119.0,4,Девятаев
4,176549,4151,91167.0,5,Секреты семейной жизни
5,176549,3734,74803.0,6,Прабабушка легкого поведения
6,176549,2657,68581.0,7,Подслушано
7,176549,4880,55043.0,8,Афера
8,176549,142,45367.0,9,Маша
9,176549,6809,40372.0,10,Дуров


In [54]:
pop_mean_weights.head(10)

,user_id,item_id,score,rank,title
0,176549,1714,635143.666667,1,Люби меня таким
1,176549,3642,442423.000000,2,Обручальное кольцо
2,176549,856,400153.000000,3,Приют комедиантов
3,176549,3480,373410.666667,4,Жизнь Мехэк
4,176549,16216,340005.500000,5,Анна-детективъ
5,176549,13395,339193.000000,6,Крамола
6,176549,11566,328638.566667,7,Татьянин день
7,176549,1914,294688.000000,8,Неравный брак
8,176549,2788,290355.500000,9,Ганга
9,176549,3750,285721.000000,10,Юлия Высоцкая


## Metrics

In [67]:
metrics = {
    "prec@5": Precision(k=5),
    "recall@5": Recall(k=5),
    "MAP@5": MAP(k=5),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "MAP@20": MAP(k=20),
    "prec@20": Precision(k=20),
    "recall@20": Recall(k=20),
    "MAP@100": MAP(k=100),
    "prec@100": Precision(k=100),
    "recall@100": Recall(k=100),
    "MAP@100": MAP(k=100),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

In [ ]:
catalog = train[Columns.Item].unique()
metric__pop = calc_metrics(metrics, pop, test, train, catalog)
metric_pop_mean_weights = calc_metrics(metrics, pop_mean_weights, test, train, catalog)

In [ ]:
metric_values_pop

In [ ]:
metric_values_pop_mean_weight